In [56]:
import pandas as pd

In [57]:
def leer_datos_por_año(archivo, columna_estado, columnas):
    datos_por_año = {}
    xls = pd.ExcelFile(archivo)
    
    datos_por_año=pd.DataFrame(columns=columnas)
    for hoja in xls.sheet_names:
        df = pd.read_excel(archivo, sheet_name=hoja)
        año = int(hoja)
        df['AÑO'] = año  # Agregar columna de año para identificar de qué año es cada registro
        
        # Crear la columna 'PEN_ALIM' con valores predeterminados en 'NO'
        df['PEN_ALIM'] = 'SI'
        
        datos_por_año=pd.concat([datos_por_año,df])
        #datos_por_año[año] = df  # pd stack 
    return datos_por_año
#correr solo una vez 
columnas= ['EPS','sexo', 'ESTADO', 'REGIMEN', 'JORNA', 'DEPARTAMENTO','EDAD', 'FECHAA','AÑO', 'PEN_ALIM']
df_pension_VO= leer_datos_por_año('Pensión alimenticia 2014.2022 Veracruz Oaxaca PTRI.xlsx', 'ESTADO', columnas)

In [58]:
df_pension_VO.drop(columns='EPS', inplace=True)

In [59]:
df_pension_VO

,sexo,ESTADO,REGIMEN,JORNA,DEPARTAMENTO,EDAD,FECHAA,AÑO,PEN_ALIM
0,MASCULINO,VERACRUZ,PC,3,54112,61,201412,2014,SI
1,MASCULINO,OAXACA,PS,0,53400,33,201412,2014,SI
2,MASCULINO,VERACRUZ,PS,0,54250,35,201412,2014,SI
3,MASCULINO,VERACRUZ,PS,1,47209,55,201412,2014,SI
4,MASCULINO,OAXACA,PS,1,54310,44,201412,2014,SI
...,...,...,...,...,...,...,...,...,...
7189,MASCULINO,VERACRUZ,PS,4,42330,37,201412,2022,SI
7190,MASCULINO,VERACRUZ,PS,4,42340,58,201412,2022,SI
7191,MASCULINO,VERACRUZ,PS,1,41240,49,201412,2022,SI
7192,MASCULINO,VERACRUZ,PS,0,42000,43,201412,2022,SI


# Leer plantilla laboral Oaxaca y Veracruz  y crear datos combinados con pension alimenticia 

In [60]:
df_oax=pd.read_excel('Plantilla laboral.Veracruz.Oaxaca.xlsx',sheet_name='oaxaca')
df_ver=pd.read_excel('Plantilla laboral.Veracruz.Oaxaca.xlsx',sheet_name='veracruz')
df_VO_completo= pd.concat([df_ver, df_oax])
df_VO_completo

,AÑO,FICHA,TIPO CONTRATACION,GENERO,JORNADA,NIVEL,ANTIGUEDAD_AÑOS,EDAD,DEPARTAMENTO,ESTADO
0,2014,227331,Planta Sindicalizado,Femenino,7,26,NaN,51.0,662 21000 GERENCIA DE ALMACENAMIENTO Y REPARTO...,VERACRUZ
1,2014,466381,Transitorio Sindicalizado,Femenino,1,11,NaN,26.0,517 54040 GERENCIA DE OPERACIÓN MARÍTIMA Y POR...,VERACRUZ
2,2014,151693,Planta Sindicalizado,Femenino,0,24,NaN,48.0,331 43640 DIRECCIÓN CORPORATIVA DE FINANZAS,VERACRUZ
3,2014,487133,Planta Sindicalizado,Femenino,0,22,NaN,44.0,"580 53000 GERENCIA DE TRANSPORTE, MANTENIMIENT...",VERACRUZ
4,2014,194904,Planta Confianza,Femenino,0,36,NaN,47.0,671 63620 GERENCIA DE VENTAS A ESTACIONES DE S...,VERACRUZ
...,...,...,...,...,...,...,...,...,...,...
41440,2022,403198,Planta Confianza,Masculino,0,37,17.0,40.0,4005 11823 SUPERINTENDENCIA DE GESTIÓN DEL MAN...,OAXACA
41441,2022,348217,Planta Sindicalizado,Masculino,1,12,18.0,46.0,4005 11831 SUPERINTENDENCIA DE EVALUACIÓN OPER...,OAXACA
41442,2022,396042,Planta Sindicalizado,Masculino,0,23,18.0,47.0,4005 11842 TALLER DE SOLDADURA,OAXACA
41443,2022,519933,Transitorio Confianza,Masculino,0,34,4.0,45.0,4005 11821 SUPERINTENDENCIA DE PROGRAMACIÓN DE...,OAXACA


In [61]:
# prepare to merge, modify columns, etc. 

columnas_cambio={'GENERO':'sexo', 'TIPO CONTRATACION': 'REGIMEN', 'JORNADA':'JORNA'}

df_VO_completo=df_VO_completo.rename(columns=columnas_cambio)

#cambiar departamento y agregar colunmna con descripcion 
#regex to extract middle number from df_VO_completo['DEPARTAMENTO']

df_VO_completo['DEPT DESC']=df_VO_completo['DEPARTAMENTO'].str.extract('\s\S*\s(.*)')

df_VO_completo['DEPARTAMENTO']=df_VO_completo['DEPARTAMENTO'].str.extract('^.*?\s(\w+)')

#change sexo and make it uppercase 
df_VO_completo['sexo'] = df_VO_completo['sexo'].str.upper()
#change regimen and rename with Dictionary 
plazas= {'Planta Sindicalizado':'PS' , 'Transitorio Sindicalizado':'TS',
       'Planta Confianza':'PC', 'Transitorio Confianza': 'TC'}

df_VO_completo['REGIMEN']=df_VO_completo['REGIMEN'].replace(plazas, regex=True)

df_VO_completo


,AÑO,FICHA,REGIMEN,sexo,JORNA,NIVEL,ANTIGUEDAD_AÑOS,EDAD,DEPARTAMENTO,ESTADO,DEPT DESC
0,2014,227331,PS,FEMENINO,7,26,NaN,51.0,21000,VERACRUZ,GERENCIA DE ALMACENAMIENTO Y REPARTO GOLFO
1,2014,466381,TS,FEMENINO,1,11,NaN,26.0,54040,VERACRUZ,GERENCIA DE OPERACIÓN MARÍTIMA Y PORTUARIA
2,2014,151693,PS,FEMENINO,0,24,NaN,48.0,43640,VERACRUZ,DIRECCIÓN CORPORATIVA DE FINANZAS
3,2014,487133,PS,FEMENINO,0,22,NaN,44.0,53000,VERACRUZ,"GERENCIA DE TRANSPORTE, MANTENIMIENTO Y SERVIC..."
4,2014,194904,PC,FEMENINO,0,36,NaN,47.0,63620,VERACRUZ,GERENCIA DE VENTAS A ESTACIONES DE SERVICIO
...,...,...,...,...,...,...,...,...,...,...,...
41440,2022,403198,PC,MASCULINO,0,37,17.0,40.0,11823,OAXACA,SUPERINTENDENCIA DE GESTIÓN DEL MANTENIMIENTO
41441,2022,348217,PS,MASCULINO,1,12,18.0,46.0,11831,OAXACA,SUPERINTENDENCIA DE EVALUACIÓN OPERATIVA Y SIS...
41442,2022,396042,PS,MASCULINO,0,23,18.0,47.0,11842,OAXACA,TALLER DE SOLDADURA
41443,2022,519933,TC,MASCULINO,0,34,4.0,45.0,11821,OAXACA,SUPERINTENDENCIA DE PROGRAMACIÓN DE PAROS DE P...


In [62]:
df_pension_VO

,sexo,ESTADO,REGIMEN,JORNA,DEPARTAMENTO,EDAD,FECHAA,AÑO,PEN_ALIM
0,MASCULINO,VERACRUZ,PC,3,54112,61,201412,2014,SI
1,MASCULINO,OAXACA,PS,0,53400,33,201412,2014,SI
2,MASCULINO,VERACRUZ,PS,0,54250,35,201412,2014,SI
3,MASCULINO,VERACRUZ,PS,1,47209,55,201412,2014,SI
4,MASCULINO,OAXACA,PS,1,54310,44,201412,2014,SI
...,...,...,...,...,...,...,...,...,...
7189,MASCULINO,VERACRUZ,PS,4,42330,37,201412,2022,SI
7190,MASCULINO,VERACRUZ,PS,4,42340,58,201412,2022,SI
7191,MASCULINO,VERACRUZ,PS,1,41240,49,201412,2022,SI
7192,MASCULINO,VERACRUZ,PS,0,42000,43,201412,2022,SI


In [63]:
# note of antiguedad anios y departamentos, explore these columns 
df_pension_VO["EDAD"] = pd.to_numeric(df_pension_VO["EDAD"]).astype('Int64')
df_pension_VO['AÑO'] = pd.to_numeric(df_pension_VO['AÑO']).astype('Int64')
df_pension_VO['JORNA'] = pd.to_numeric(df_pension_VO['JORNA']).astype('Int64')
df_pension_VO['DEPARTAMENTO'] = pd.to_numeric(df_pension_VO['DEPARTAMENTO']).astype('Int64')
df_VO_completo["DEPARTAMENTO"]=pd.to_numeric(df_VO_completo['DEPARTAMENTO']).astype('Int64')


datos_combinados_VO=pd.merge(df_VO_completo,df_pension_VO,  on=['AÑO', 'REGIMEN', 'sexo', 'JORNA', 'EDAD', 'DEPARTAMENTO', 'ESTADO'], how='outer', indicator=True)

In [64]:
datos_combinados_VO._merge.value_counts()

_merge
left_only     101821
both           81366
right_only     25276
Name: count, dtype: int64

In [65]:
datos_combinados_VO[datos_combinados_VO['_merge']=='both']

,AÑO,FICHA,REGIMEN,sexo,JORNA,NIVEL,ANTIGUEDAD_AÑOS,EDAD,DEPARTAMENTO,ESTADO,DEPT DESC,FECHAA,PEN_ALIM,_merge
54,2014,225630.0,PC,FEMENINO,0,37.0,NaN,46.0,54440,VERACRUZ,GERENCIA DE OPERACIÓN MARÍTIMA Y PORTUARIA,201412,SI,both
167,2014,399128.0,PC,MASCULINO,0,30.0,NaN,31.0,59110,VERACRUZ,DIRECCIÓN GENERAL DE PEMEX LOGÍSTICA,201412,SI,both
181,2014,428841.0,PC,MASCULINO,0,37.0,NaN,33.0,21260,VERACRUZ,GERENCIA DE ALMACENAMIENTO Y REPARTO GOLFO,201412,SI,both
186,2014,346110.0,PC,MASCULINO,0,35.0,NaN,33.0,47200,VERACRUZ,"GERENCIA DE REFINERÍA MINATITLAN, VER.",201412,SI,both
192,2014,374405.0,PC,MASCULINO,0,33.0,NaN,34.0,43640,VERACRUZ,DIRECCIÓN CORPORATIVA DE FINANZAS,201412,SI,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185355,2021,359951.0,TS,MASCULINO,7,13.0,0.0,61.0,11840,OAXACA,PATIO Y MANIOBRAS,202112,SI,both
185358,2021,498722.0,TS,MASCULINO,7,11.0,13.0,61.0,11849,OAXACA,TALLER DE CARPINTERÍA,202112,SI,both
185360,2021,400713.0,TS,MASCULINO,7,8.0,6.0,62.0,11801,OAXACA,SUPERINTENDENCIA DE INSPECCIÓN TÉCNICA,202112,SI,both
185364,2021,946649.0,TS,MASCULINO,7,16.0,5.0,62.0,11847,OAXACA,TALLER DE ALBAÑILERÍA,202112,SI,both


In [66]:
datos_combinados_VO[datos_combinados_VO.duplicated()==True]

,AÑO,FICHA,REGIMEN,sexo,JORNA,NIVEL,ANTIGUEDAD_AÑOS,EDAD,DEPARTAMENTO,ESTADO,DEPT DESC,FECHAA,PEN_ALIM,_merge
284,2014,465812.0,PC,MASCULINO,0,35.0,NaN,40.0,47300,OAXACA,"GERENCIA DE REFINERÍA SALINA CRUZ, OAX.",201412,SI,both
286,2014,371630.0,PC,MASCULINO,0,35.0,NaN,40.0,47300,OAXACA,"GERENCIA DE REFINERÍA SALINA CRUZ, OAX.",201412,SI,both
301,2014,200099.0,PC,MASCULINO,0,38.0,NaN,41.0,42100,VERACRUZ,"GERENCIA DE REFINERÍA MINATITLAN, VER.",201412,SI,both
307,2014,NaN,PC,MASCULINO,0,NaN,NaN,41.0,44100,VERACRUZ,NaN,201412,SI,right_only
309,2014,NaN,PC,MASCULINO,0,NaN,NaN,41.0,44210,VERACRUZ,NaN,201412,SI,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208310,2022,NaN,TS,MASCULINO,7,NaN,NaN,46.0,46240,OAXACA,NaN,201412,SI,right_only
208345,2022,NaN,TS,MASCULINO,7,NaN,NaN,49.0,54112,VERACRUZ,NaN,201412,SI,right_only
208346,2022,NaN,TS,MASCULINO,7,NaN,NaN,49.0,54112,VERACRUZ,NaN,201412,SI,right_only
208367,2022,NaN,TS,MASCULINO,7,NaN,NaN,51.0,47208,VERACRUZ,NaN,201412,SI,right_only


In [67]:
datos_combinados_VO.drop_duplicates(inplace=True)

In [68]:
datos_combinados_VO

,AÑO,FICHA,REGIMEN,sexo,JORNA,NIVEL,ANTIGUEDAD_AÑOS,EDAD,DEPARTAMENTO,ESTADO,DEPT DESC,FECHAA,PEN_ALIM,_merge
0,2014,527430.0,PC,FEMENINO,0,30.0,NaN,26.0,63640,VERACRUZ,GERENCIA DE VENTAS A ESTACIONES DE SERVICIO,NaN,NaN,left_only
1,2014,560424.0,PC,FEMENINO,0,30.0,NaN,28.0,63640,VERACRUZ,GERENCIA DE VENTAS A ESTACIONES DE SERVICIO,NaN,NaN,left_only
2,2014,507590.0,PC,FEMENINO,0,30.0,NaN,29.0,59140,VERACRUZ,GERENCIA DE ALMACENAMIENTO Y SERVICIOS PORTUARIOS,NaN,NaN,left_only
3,2014,547943.0,PC,FEMENINO,0,30.0,NaN,29.0,63640,VERACRUZ,GERENCIA DE VENTAS A ESTACIONES DE SERVICIO,NaN,NaN,left_only
4,2014,518018.0,PC,FEMENINO,0,33.0,NaN,30.0,42200,VERACRUZ,"GERENCIA DE REFINERÍA MINATITLAN, VER.",NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208458,2022,NaN,TS,MASCULINO,7,NaN,NaN,68.0,47203,VERACRUZ,NaN,201412,SI,right_only
208459,2022,692788.0,TS,MASCULINO,7,21.0,7.0,69.0,11844,OAXACA,TALLER DE TUBERÍA,NaN,NaN,left_only
208460,2022,762229.0,TS,MASCULINO,7,11.0,1.0,69.0,11849,OAXACA,TALLER DE CARPINTERÍA,NaN,NaN,left_only
208461,2022,921901.0,TS,MASCULINO,7,11.0,1.0,70.0,11838,OAXACA,TALLER ELÉCTRICO,NaN,NaN,left_only


In [69]:
datos_combinados_VO._merge.value_counts()

_merge
left_only     101821
both           52316
right_only     19275
Name: count, dtype: int64

In [70]:
datos_combinados_VO['PEN_ALIM'].fillna('NO', inplace=True)

/var/folders/dd/w9fvb4q53yl3mvzg47fqvxrr0000gn/T/ipykernel_28973/2005451639.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos_combinados_VO['PEN_ALIM'].fillna('NO', inplace=True)


In [71]:
datos_combinados_VO['PEN_ALIM'].value_counts()

PEN_ALIM
NO    101821
SI     71591
Name: count, dtype: int64

In [72]:
datos_combinados_VO["DEPARTAMENTO"]=datos_combinados_VO['DEPARTAMENTO'].astype(str) #back to object 

#drop columns you don't need 
datos_combinados_VO.drop(columns=['FICHA', '_merge'], inplace=True)

In [73]:
datos_combinados_VO

,AÑO,REGIMEN,sexo,JORNA,NIVEL,ANTIGUEDAD_AÑOS,EDAD,DEPARTAMENTO,ESTADO,DEPT DESC,FECHAA,PEN_ALIM
0,2014,PC,FEMENINO,0,30.0,NaN,26.0,63640,VERACRUZ,GERENCIA DE VENTAS A ESTACIONES DE SERVICIO,NaN,NO
1,2014,PC,FEMENINO,0,30.0,NaN,28.0,63640,VERACRUZ,GERENCIA DE VENTAS A ESTACIONES DE SERVICIO,NaN,NO
2,2014,PC,FEMENINO,0,30.0,NaN,29.0,59140,VERACRUZ,GERENCIA DE ALMACENAMIENTO Y SERVICIOS PORTUARIOS,NaN,NO
3,2014,PC,FEMENINO,0,30.0,NaN,29.0,63640,VERACRUZ,GERENCIA DE VENTAS A ESTACIONES DE SERVICIO,NaN,NO
4,2014,PC,FEMENINO,0,33.0,NaN,30.0,42200,VERACRUZ,"GERENCIA DE REFINERÍA MINATITLAN, VER.",NaN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
208458,2022,TS,MASCULINO,7,NaN,NaN,68.0,47203,VERACRUZ,NaN,201412,SI
208459,2022,TS,MASCULINO,7,21.0,7.0,69.0,11844,OAXACA,TALLER DE TUBERÍA,NaN,NO
208460,2022,TS,MASCULINO,7,11.0,1.0,69.0,11849,OAXACA,TALLER DE CARPINTERÍA,NaN,NO
208461,2022,TS,MASCULINO,7,11.0,1.0,70.0,11838,OAXACA,TALLER ELÉCTRICO,NaN,NO


In [42]:
import os.path

if not os.path.exists('Datos_Pension_Combinados_VO.csv'): 
    datos_combinados_VO.to_csv('Datos_Pension_Combinados_VO.csv')
    

In [53]:
datos_combinados_VO.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154137 entries, 0 to 183186
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   AÑO              154137 non-null  int64  
 1   REGIMEN          154137 non-null  object 
 2   sexo             154137 non-null  object 
 3   JORNA            154137 non-null  int64  
 4   NIVEL            154137 non-null  int64  
 5   ANTIGUEDAD_AÑOS  92859 non-null   float64
 6   EDAD             154121 non-null  float64
 7   DEPARTAMENTO     154137 non-null  object 
 8   ESTADO           154137 non-null  object 
 9   DEPT DESC        154136 non-null  object 
 10  FECHAA           52316 non-null   object 
 11  PEN_ALIM         154137 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 15.3+ MB


In [75]:
datos_combinados_VO[datos_combinados_VO['AÑO']==2022]['PEN_ALIM'].value_counts()

PEN_ALIM
NO    15879
SI     5742
Name: count, dtype: int64